# Lab 2: Step Response of an RC Circuit

## Introduction

This lab has three primary learning objectives:

1. helping students learn how to perform real-time dynamic systems and
   control experiments using the Dr. Krauss' `wxbd_gui` and `py_block_diagram` modules
2. helping students learn about transfer functions through
   investigating the step response of an RC circuit
3. helping students learn to curve fit data using `scipy.optimize.fmin`

Throughout this course, we will use Arduino microcontrollers in conjunction with Python to run dynamic systems and control experiments.  

An RC circuit is a fairly simple dynamic system and will be used in this lab to help students begin to think in terms of transfer functions.  The results of an experimental step response will be used to estimate transfer function coefficients.  Experimental results will then be compared to Python simulations.


## Lab 2 Prerequisites 
- install wxPython
- install Dr Krauss' python libraries
- install Arduino IDE 1.8.x
- install Dr Krauss' Arduino Libraries:
    - rtblockdiagram and kraussserial
- launch `wxbd_gui`


### install wxPython

- Windows: pip install wxPython
    - jupyter or anaconda prompt
- Mac terminal: pip3 install wxPython


### install Dr Krauss' python libraries
	
- Windows: pip install wxbd_gui
- Mac: pip3 install wxbd_gui

### Arduino Stuff

- install Arduino IDE 1.8.x
- install Dr Krauss' Arduino Libraries: rtblockdiagram and kraussserial
    - Windows: download zip files from GitHub
        - [how to install Arduino libraries from zip files](https://docs.arduino.cc/software/ide-v1/tutorials/installing-libraries/#importing-a-zip-library)
    - Mac: git clone in terminal from Arduino libraries folder:

```
cd ~/Arduino/libraries/
git clone https://github.com/ryanGT/rtblockdiagram.git
git clone https://github.com/ryanGT/kraussserial.git
```

**Mac Note:** Both `git clone` commands need to be executed within the folder `~/Arduino/libraries/`.

**Dr. Krauss' git repos for Arduino:**

- [https://github.com/ryanGT/rtblockdiagram](https://github.com/ryanGT/rtblockdiagram)
-  [https://github.com/ryanGT/kraussserial](https://github.com/ryanGT/kraussserial)

### Launch wxbd_gui on your computer 
	
- Windows: double click on launch_wxbd_gui
    - you may need to know how to tell windows where python.exe is
- Mac: navigate to the 345_lab_git/scripts_ 345 folder and execute the command: `python3 launch_wxbd_gui`




## Main Steps

- install and use Dr. Krauss' `wxbd_gui` module to auto-generate Arduino code 
  the performs a step response test for an RC circuit
- build an RC circuit and connect it to the Arduino
- compile the auto-generated Arduino code so that a step response test is run and 
  data is printed to the serial monitor
- overlay the experimental step response with the result of Laplace analysis "by hand" as well as simulation results using `control.step_response`
- curve fit the experimental step response data to improve your estimate of the TF parameters

## Part 1: Installing and Using Dr. Krauss' `wxbd_gui` to Model the Dynamic System

## Arduino Template

`wxbd_gui` uses an Arduino template and inserts code for the block diagram model and certain points in the template.  The template is in `345_lab_git` and is called `arduino_template1_rev_3.ino`.  You will need to tell `wxbd_gui` where to find it. 

Note that the template cannot be compiled and is not intended to be compiled.  You also need to tell `wxbd_gui` the location of a folder where you want the output code saved.  That code will be compiled and loaded onto an Arduino Uno.

Remember that Arduino code is saved in its own folder where the folder name is the same as the final `*.ino` file.  You give `wxbd_gui` the folder location and it creates the matching `*.ino` file with the correct name in the folder.  For example, if you gave it the folder location `C:\Users\DrKrauss\Documents\lab1\RC_ino_1`, it would create a file called `RC_ino_1.ino` and place it in that folder.

## Part 2: RC Step Response

Once you have created the system model in `wxbd_gui` with a step input connected to the plant, you are ready to auto-generate the Arduino code and compile it.  Once the code compiles correctly, you are ready to connect the RC circuit below and run a test.  Real-time data should print to the serial monitor. 

## Final Block Diagram

<img src="figs/RC_bd_screenshot.png" width=500px>

## Step Response Testing

<img src="figs/rc_schematic.png" width=400px>

A schematic of an RC filter is shown above.
In order to run a step response test, you will need to modify the
Python code from Part 1 to communicate with the Arduino over serial.


## Saving Your Data

Once you have successfully run an RC step response test, save that data to a CSV or TXT file before you start curve fitting.  This will allow you to work on curve fitting without having to have your Arduino attached and running a new test each time.

In order to save the data, copy the data from the Arduino serial monitor and paste it into a text editor.  Please note that Microsoft Word and Google Docs are NOT text editors.  We want to save the data into a file that contains only plain text.

You will know you have done this correctly if you can use `np.loadtxt` to load the data into Python and plot it like you did in lab 1.


## Curve Fitting

Once you have the step response data, use `scipy.optimize.fmin` to
curve fit it.  In order to do this, you will need to create a function that returns $v_{out}$ for a 5V step input.  First, find the transfer function of the circuit

$$G(s) = \frac{V_{out}(s)}{V_{in}(s)}$$

We found the transfer function in lecture.  Verify the transfer
function we found in lecture using Kirchoff's laws, Ohm's law, and the
voltage/current relationship of a capacitor.  Once you have verified
the transfer function, write a function that finds the step response
with a 5V input.  You ought to be able to modify code from lab 1 to
create the function you need for curve fitting.


## optimize.fmin Example

A Jupyter notebook example has been created to introduce curve fitting in Python based on the function `scipy.optimize.fmin`.  The example requires a `csv` data file.  The notebook and the example (fake) data file are in a subfolder called `helper_notebooks` under that lab 2 git folder.

### Curve Fitting Tips

In order to fit a curve to the step response using `optimize.fmin`, you need a function that takes the unknown transfer function coefficients (i.e. $p$ in this case) and returns the step response.  This is very similar to what we did in lab 1 except that we need to find the step response at the same points in time where we have experimental data.  There are two options for doing this:

1. Solve the step response symbolically with $p$ as a variable and then just have your `mymodel` function use that symbolic expression
2. Pass in your time array as the second input to `control.step_response`

Option 2 is in some ways easier but comes with one additional
challenge: `control.step_response` expects the time vector to have
perfect time steps that are exactly the same.  Essentially, it is expecting that your time vector was created using `np.arange`.  So, you need to find a way to create a fake, perfect time vector that is as close as possible to your real, experimental time vector.  Find the average time step `dt` from your experimental time vector and use it along with `np.arange` to create a fake time vector that has the same length as the experimental time vector.

Note that in Python, functions always have read-only access to all of
the variables in the notebook or Python file where they reside.  So,
you need to pass your fake time vector as a second input to
`control.step_response`, but it does not need to be an input to the
`mymodel` function.  All you need to do is define the fake time vector
somewhere in the notebook.  The only input to `mymodel` should be a
list of unknown coefficients, `[p]`.


## Step Response Simulation

You are required to compare your experimental results to a simulation
of the step response using either `control.step_response` or
`control.forced_response` from the Python `control` module.  You will
need the transfer function of the system to do this.  To show your
results, overlay data from your experimental step response on the same
plot as simulation results based on your optimized coefficient values.
Also, verify that the step response from the `control` module agrees with
your derivation for $v_{out}(t)$ for the step input.


# Comprehension Questions

### CQ1: step response

What steps are necessary to perform step response analysis "by hand"?  How do you do the same steps using Python to verify your answer?

### CQ2: optimize.fmin

What does `optimze.fmin` actually do?  What is the scalar value returned by the cost function?  